### <span style='color:green'>**Optional Prepare Step**</span>: Update Notebook to Latest Version
Use this notebook if you want to update any other notebook to the latest version.
Please note that the files a copied from the curent development source directory which can be work in progress.
Future versions of this script will update scripts from the latest version in GitHub.
<br>
Please stop and close any open notebooks that you want to update. You can shutdown running notebooks in the Jupyter dashboard "Running" tab.

** TODO: Print timestamp or warning that notebook is outdated **

In [44]:
# Code: Update WTI master data from Google Drive

import os
os.sys.path = os.sys.path if '..' in os.sys.path else os.sys.path + ['..']

import shutil
import glob
import logging
import zipfile
import time
import ipywidgets as widgets 

from common.widgets_utility import BaseWidgetUtility

root_source_folder = '/home/roger/notebooks/the_culture_of_international_relations'

source_folders = [
    '1_quantitative_analysis',
    '2_network_analysis',
    '3_text_analysis'
]
    
logging.getLogger().setLevel(logging.INFO)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def ts_data_path(self, filename):
    return os.path.join(self.directory, '{}_{}'.format(time.strftime("%Y%m%d%H%M"), filename))

def data_path_ts(self, path):
    basename, extension = os.path.splitext(path)
    return os.path.join(self.directory, '{}_{}{}'.format(basename, time.strftime("%Y%m%d%H%M"), extension))

def get_notebook_filenames(folder, extension='ipynb'):
    '''  Returns a list of files found in given folder '''
    return [ os.path.split(x)[1] for x in glob.glob(os.path.join(folder, '*.{}'.format(extension))) ]

def get_sourcefiles_to_archive(folder='../*/**/', extensions='ipynb;py'):
    '''  Returns a list of files found in given folder '''
    filenames = []
    for extension in extensions.split(';'):
        filenames += glob.iglob(os.path.join(folder, '*.' + extension))
    return filenames

def archive_sourcefiles():
    target_file = './backup/{}_source_code.zip'.format(time.strftime("%Y%m%d%H%M"))
    with zipfile.ZipFile(target_file, "w", zipfile.ZIP_DEFLATED) as zip:
        for source_file in get_sourcefiles_to_archive():
            zip.write(source_file, os.path.basename(source_file))
    return target_file

def copy_source_files_in_folder(source_folder, destination_folder, pattern):
    for filename in glob.glob(os.path.join(source_folder, pattern)):
        try:
            shutil.copy(filename, destination_folder)
            print("Updated {}".format(os.path.split(filename)[1]))
        except shutil.SameFileError as ex:
            print("Unchanged: {} (skipped)".format(os.path.split(filename)[1]))
    print('Done updating folder {}'.format(os.path.split(destination_folder)[1]))

def update_notebook(folder, confirm, root_folder, copy_python_files=True):
    global upw
    if folder is None:
        return
    
    if confirm is False:
        print('Please confirm update by checking the CONFIRM button!')
        return
    
    # target_archive = archive_sourcefiles()
    # print('  Current version stored in {}.'.format(target_archive))
    
    copy_source_files_in_folder(os.path.join(root_folder, folder), os.path.join('../', folder), '*.py')
    copy_source_files_in_folder(os.path.join(root_folder, folder), os.path.join('../', folder), '*.ipynb')
    
    if copy_python_files:
        copy_source_files_in_folder(os.path.join(root_folder, "common"), '../common', '*.py')
        print('  Python source files updated.')
        
    upw.folder.value = None
    upw.confirm.value = False
    

upw = BaseWidgetUtility(
    folder=widgets.Dropdown(
        options=source_folders,
        value=None,
        description='Folder:',
        layout=widgets.Layout(width='400px')
    ),
    confirm=widgets.ToggleButton(
        description='Confirm',
        button_style='',
        icon='check',
        value=False
    ),)
iupw = widgets.interactive(update_notebook, folder=upw.folder, confirm=upw.confirm, root_folder=root_source_folder)
display(widgets.VBox([widgets.HBox([upw.folder, upw.confirm]), iupw.children[-1]]))
# iupw.update()

